In [24]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
import json
from langchain.agents import AgentExecutor, Tool
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate

from langchain.memory import ConversationBufferMemory

from langchain.agents import AgentExecutor, create_structured_chat_agent

from google.cloud import firestore
from langchain_google_firestore import FirestoreChatMessageHistory

In [25]:
### This notebook test store the conversation history on clould (firestore)

In [26]:
load_dotenv()

True

In [27]:
llm = ChatOpenAI(model="gpt-4o",temperature=0.1)

In [28]:
firestore_project_id = "littleseven"
firestore_session_id = "yiding" 
firestore_collection_name = "history" # use a new name to start a new conversation

In [29]:
client = firestore.Client(project=firestore_project_id)

In [30]:
# Initialize Firestore Client
client = firestore.Client(project=firestore_project_id)

In [31]:
chat_history = FirestoreChatMessageHistory(
    session_id=firestore_session_id,
    collection=firestore_collection_name,
    client=client,
)

In [32]:
chat_history.messages

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hey there! How's it going?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is time now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The current local time is 6:44 PM.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='exit.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Alright, if you need anything else, just let me know. Have a great day!', additional_kwar

In [10]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_28082/1976451665.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
memory.chat_memory.messages

[]

In [12]:
for message in chat_history.messages:
    if message.type == "human":
        memory.chat_memory.add_user_message(message.content)
    elif message.type == "ai":
        memory.chat_memory.add_ai_message(message.content)

In [13]:
memory.chat_memory.messages

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hey there! How's it going?", additional_kwargs={}, response_metadata={})]

In [14]:
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [15]:
tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time."
    )
]

In [16]:
system_message=SystemMessagePromptTemplate.from_template(
    """ You are an AI assistant. Your responses will be converted into speech.
        Please ensure that your answers:
        - Are conversational and sound natural when spoken.
        - Use a friendly and engaging tone.
        - Avoid overly formal or robotic language.
        - Use contractions where appropriate (e.g., "I'm" instead of "I am").
        - Keep sentences short and easy to understand.
        - Use a storytelling style when possible.
        
        Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
        """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [17]:
prompt_template=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [18]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt_template)

In [19]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory,handle_parsing_errors=True, verbose=True)

In [20]:
memory.chat_memory.messages

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hey there! How's it going?", additional_kwargs={}, response_metadata={})]

In [21]:
# response = agent_executor.invoke({
#     "input": "what is the time now?",
# })

# print(response)

In [22]:
# response['output']

In [23]:
while True:
    user_msg = input("User: ")
    if user_msg.lower() == "exit":
        break

    chat_history.add_user_message(user_msg)

    response=agent_executor.invoke({"input":user_msg})
    chat_history.add_ai_message(response["output"])

    print(f"AI: {response['output']}")



> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "Hello! How can I assist you today?"
}

> Finished chain.
AI: Hello! How can I assist you today?


> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "I'm doing great, thanks for asking! How about you?"
}

> Finished chain.
AI: I'm doing great, thanks for asking! How about you?


> Entering new AgentExecutor chain...
Thought: I need to find out the current local time to answer the question.
Action:
```json
{
  "action": "LocalTime",
  "action_input": ""
}
```Current local time is: 18:44:45Action:
```json
{
  "action": "Final Answer",
  "action_input": "The current local time is 6:44 PM."
}
```

> Finished chain.
AI: The current local time is 6:44 PM.


> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "Alright, if you need anything else, just let me know. Have a great day!"
}

> Finished chain.
AI: Alright, if you need anything e